In [1]:
import torch
torch.set_printoptions(precision=15)
from torch import nn
import numpy as np
import sys
sys.path.append('../fairseq-l-system-captioning/')

from lsystem_modules import SlimLstm

vocab = torch.load('../fairseq-l-system-captioning/data/Advanced_47946_10_max_derivations_vocab.pth')

model = SlimLstm.load_from_checkpoint('../fairseq-l-system-captioning/checkpoints/SlimLstm_normal_rand/epoch=493-step=670852.ckpt')

/home/magnusson/miniconda3/envs/torch/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [2]:
test_cell = nn.LSTMCell(256,256)

In [3]:
test_cell.weight_ih = model.decoder.lstm.weight_ih_l0
test_cell.weight_hh = model.decoder.lstm.weight_hh_l0
test_cell.bias_ih   = model.decoder.lstm.bias_ih_l0
test_cell.bias_hh   = model.decoder.lstm.bias_hh_l0

In [4]:
x = torch.rand((3,256)).double()
x.requires_grad = True

In [5]:
from lrp.functional.lstm import LstmAll

In [6]:
test_cell.eps = torch.tensor(0.001)
test_cell.bias_factor = torch.tensor(0.0)

a = torch.zeros(256).double()
outs = torch.zeros_like(x)
out_a, h_a, c_a = LstmAll.apply(x[0, None], a[None], a[None], test_cell)
outs[0] = out_a
out_a, h_a, c_a = LstmAll.apply(x[1, None], h_a, c_a, test_cell)
outs[1] = out_a
out_a, h_a, c_a = LstmAll.apply(x[2, None], h_a, c_a, test_cell)
outs[2] = out_a

In [7]:
# h_a.sum().backward()
(out_a.sum() / 256).backward()

In [8]:
from lrp.functional._lstm import LSTM_bidi

lstm = LSTM_bidi(model.decoder.lstm)

In [9]:
Rx = lstm.lrp(x.detach().numpy())

In [10]:
torch.testing.assert_close(out_a, torch.tensor(lstm.s)[None])

In [11]:
R_x = x.grad
torch.testing.assert_close(R_x, torch.tensor(Rx))

In [12]:
from lrp.lstm import LSTM

l = LSTM.from_torch(model.decoder.lstm)


In [13]:
i = x.detach()
i.requires_grad = True
out, (h, c) = l.forward(i, explain=True)

torch.testing.assert_close(out, outs)
torch.testing.assert_close(out[-1, None], torch.tensor(lstm.s)[None])

In [14]:
(out.sum() / 256).backward()

In [15]:
R_x = i.grad
torch.testing.assert_close(R_x, torch.tensor(Rx))